# Notebook for Without Social Score

In [99]:
import datetime
import json
import matplotlib.pyplot as plt
import mysql.connector
import numpy as np
import requests
import csv
from mysql.connector import errorcode
from urllib.request import URLError, Request, urlopen



## Uncomment and run this cell just once to update the distance and time obtained from GraphHopper from source to destination into the database 

### (Only the first time for the respective trips)
#### command to uncomment the whole cell: (ctrl + /)

In [100]:
# #Looping through all the data in CSV and inserting them into the database
# dataCount = 0
# cnn = mysql.connector.connect(
#         user='root',
#         password='root',
#         host='localhost',
#         port=3306,
#         database='cs581_rideshare')
# cursor = cnn.cursor()

# trip_date = datetime.date(2015,5,15)
# start_time = datetime.time(8,0,0)
# end_time = datetime.time(9,0,0)


# sql1 = "select trip_id, dropoff_latitude, dropoff_longitude from trip_details2 where trip_date = '" + str(trip_date) + "'and pickup_time >= '" + str(start_time) + "'and pickup_time < '" + str(end_time) + "'"
# cursor.execute(sql1)
# result = cursor.fetchall()
# try:
#     for row in result:
#         #print(type(data[0]))
#         if float(row[1]) != 0 and float(row[2]) != 0:
#             result1 = GraphHopperDistance.find_distance(40.796842, -73.949662,row[1], row[2])
#             if result1[0] >= 0.0 and result1[1] >= 0.0:
#                 sql = "update trip_details2 set distance = %s, travel_time = %s where trip_id = %s" % (float(result1[0]), float(result1[1]), row[0])
#                 #print sql
#                 cursor.execute(sql)
#                 cnn.commit()
#             else:
#                 sql = "update trip_details2 set distance = %s, travel_time = %s where trip_id = %s" % (float(200), float(300), row[0])
#                 #print sql
#                 cursor.execute(sql)
#                 cnn.commit()
#         #print data
#         rowFlag = True
#         dataCount += 1
# except mysql.connector.Error:
#     print("error")
# print ('Rows inserted: ',dataCount)
# cursor.close()
#cnn.close()

**Setting the source lat and long position to JFK airport**

In [101]:
# JFK : (40.796842, -73.949662)
# Central Park : (40.796842, -73.949662)

(SOURCE_LATITUDE, SOURCE_LONGITUDE) = (40.796842, -73.949662)

**This function helps calculate the next pool window given the present start time and POOL WINDOW time in seconds**

In [102]:

def time_plus(time, timedelta):
    start = datetime.datetime(
        2000, 1, 1,
        hour=time.hour, minute=time.minute, second=time.second)
    end = start + timedelta
    return end.time()


    


### Set the start date, start time, end time and pool window size in seconds

In [103]:
#SETTING START DATE and START TIME and END TIME for that day
start_date = datetime.date(2015,5,15)
start_time = datetime.time(8,10,0)
until_time = datetime.time(8,11,0)

#POOL WINDOW SIZE
windowsize = datetime.timedelta(0,60) # Setting the window size
end_time = time_plus(start_time, windowsize)

**Classes to represent the Location longitude and latitude and Trip details**

In [104]:
class Location:
    """
    Represents the coordinates of a location
    """
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude
        
        
class Trip:
    """
    Represents the attributes of a trip
    """
    def __init__(self, trip_id, pickup_lat, pickup_long,dropoff_lat, dropoff_long, pickup_time, dropoff_time, passenger_count, distance,graph_distance,graph_time):
        self.id = trip_id
        self.pickup_lat = pickup_lat
        self.pickup_long = pickup_long
        self.dropoff_lat = dropoff_lat
        self.dropoff_long = dropoff_long
        self.pickup_time = pickup_time
        self.dropoff_time = dropoff_time
        self.passenger_count = passenger_count
        self.distance = distance
        self.gh_distance = graph_distance
        self.gh_time = graph_time

**Function to get all the trip details from the database for a particular trip window**

In [105]:
def get_data (start_date,start_time,end_time):
    cnn = mysql.connector.connect(
        user='root',
        password='root',
        host='localhost',
        port=3306,
        database='cs581_rideshare')
    sql = "select * from trip_details2 where trip_date = '" + str(start_date) + "'and pickup_time >= '" + str(start_time) + "'and pickup_time < '" + str(end_time) + "'"
    
    cursor = cnn.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    tripList = []
    for row in result:
        trip = Trip(int(row[0]), row[6],row[7],row[8],row[9], row[2], row[3], row[4], row[5],row[10],row[11])
        tripList.append(trip)
    cursor.close()
    cnn.close()
    return tripList

**Class with functions to calculate the distance and time by calling GraphHopper API and converting the distance and time to miles and minutes  respectively**

In [106]:
class GraphHopperDistance:
    def distance_miles(distance):
        miles = distance / 1609.344
        return round(miles, 2)

    def time_mins(time):
        minute, reminder = divmod(time, 60000)
        if reminder >= 50:
            minute += 1
        return minute

    def find_distance(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude):
        request_str = 'http://localhost:8989/route?point=' + str(pickup_latitude) + '%2C' + str(pickup_longitude) + '&point=' + str(
            dropoff_latitude) + '%2C' + str(dropoff_longitude) + '&vehicle=car'
        request = Request(request_str)

        try:
            response = urlopen(request)
            req = requests.get(request_str)
            output = json.loads(req.text)
            paths = output["paths"]
            distance = GraphHopperDistance.distance_miles(paths[0]["distance"])
            time = GraphHopperDistance.time_mins(paths[0]["time"])
            result = [distance, time]
        except URLError:
            result = [-1, -1]

        return result



**Function to obatin all the trip details given a trip id**

In [107]:
def get_trip_details (trip_id):
        cnn = mysql.connector.connect(
        user='root',
        password='root',
        host='localhost',
        port=3306,
        database='cs581_rideshare')
        
        sql = "select * from trip_details2 where trip_id = %s" %trip_id
        cursor = cnn.cursor()
        cursor.execute(sql)
        result = cursor.fetchall()
        trip = Trip(int(result[0][0]),result[0][6], result[0][7],result[0][8], result[0][9], result[0][2], result[0][3], result[0][4], result[0][5],result[0][10],result[0][11])
        cursor.close()
        cnn.close()
        return trip



## Class with functions to
    1. Create trip pairs
    2. Adding a trip to another
    4. Find the Distace Saving
    5. Find Cost Saving
    6. Find the count of trips in a merge
     

In [108]:
class Mergedtrips:
    def __init__(self, trip1, trip2, ):
        self.trip_list1 = []
        if(trip1 == trip2):
            raise Exception("Duplicate Trip")
        self.trip_list1.append(trip1)
        self.trip_list1.append(trip2)
    
    def add(self, trip_id):
        if(not(trip_id in self.trip_list1)):
            self.trip_list1.append(trip_id)
        else:    
            raise Exception("Duplicate Trip")
        
    
    def saved_distance(self,distance_matrix,dm_individual):
        #try:
        dist_matrix= distance_matrix
        dm_individual = dm_individual
        dist_m = {}
        ordered_trip_list1 = []
        for id in self.trip_list1:
            trip = get_trip_details(id)
            distance = dm_individual[trip.id]
            if distance > 0:
                dist_m[distance] = id
        for distance in sorted(list(dist_m)):
            ordered_trip_list1.append(dist_m[distance])
        trip = get_trip_details(ordered_trip_list1[0])
        combined_trip_distance = trip.gh_distance
        i=1
        source = 0
        while(i<len(ordered_trip_list1)):
            trip1 = get_trip_details(ordered_trip_list1[source])
            trip2 = get_trip_details(ordered_trip_list1[i])
            x = trip1.id
            y = trip2.id
            if x >=y:
                difference = dist_matrix[y][x]
            else:
                difference = dist_matrix[x][y]
            if difference > 0:
                source = i
                combined_trip_distance = float(combined_trip_distance) + float(difference)
            else: 
                print("error in distance gain")
            i += 1
        normal_trip_distance = sum(list(dist_m))
        return (normal_trip_distance, combined_trip_distance)
        
    def saved_cost(self,dm_individual):
        dist_m = {}
        dm_individual = dm_individual
        combined_cost = 0.0
        for id in self.trip_list1:
            trip = get_trip_details(id)
            distance = dm_individual[trip.id]
            dist_m[distance] = id
        list_mapping = list(dist_m)
        sum_distance = sum(list_mapping)
        if len(self.trip_list1) == 4:
            combined_cost = sum_distance * 0.6 
        elif len(self.trip_list1) == 3:
            combined_cost = sum_distance * 0.7
        elif len(self.trip_list1) == 2:
            combined_cost = sum_distance * 0.8
        return(sum_distance, combined_cost)
    
    def __str__(self):
        sorted_trip_list = sorted(self.trip_list1)
        hashed_string = ""
        for trip in sorted_trip_list:
            hashed_string += str(trip).zfill(20)
        return hashed_string
    
    def __hash__(self):
        return str(self).__hash__()

    def __eq__(self, other):
        return sorted(self.trip_list1) == sorted(other.trip_list1)


### Functions to calculate Social Score
    Finds the range that the Merged trip belong to and according compares the difference in the Social Preferences to the Threshold value. If satisfied then returns True, else returns False 

In [109]:
def getAllowedDifference(sharedDistance,source):
    if source=="JFK":
        if sharedDistance >20.0:
            return 0.3
        elif sharedDistance > 9.0:
            return 0.5
        else:
            return 0.8
    elif source=="MAN":
        if sharedDistance >20.0:
            return 0.8
        elif sharedDistance >9.0:
            return 0.5
        else:
            return 0.3
    else:
        if sharedDistance >40.0:
            return 0.3
        elif sharedDistance >30.0:
            return 0.5
        else:
            return 0.8
    return


def areSociallyCompatible(tripId1,tripId2,sharedDistance,source='JFK'):
    cnn = mysql.connector.connect(
        user='root',
        password='root',
        host='localhost',
        port=3306,
        database='cs581_rideshare')
    statement = "select * from user_preferences2 where tripId in("+str(tripId1)+","+str(tripId2)+")"
    cursor = cnn.cursor()
    cursor.execute(statement)
    passengers = cursor.fetchall()
    cursor.close()
    cnn.close()
    if(len(passengers)<2):
        return False
    else:
        maxRange=getAllowedDifference(sharedDistance,source)
        total_preferences=3
        prefNum=3
        compatiblePreferencesCount=0
        totalPreferencesDiff=0;
        for i in range(prefNum,prefNum+total_preferences):
            totalPreferencesDiff+=abs(passengers[0][i]-passengers[1][i])
        if (totalPreferencesDiff/30)<maxRange:
            return True
        else:
            return False
    


### Function for suspending unmerged requests to next pool if opted for

In [110]:
def nextpoolFunc(trip_id):
    cnn = mysql.connector.connect(
        user='root',
        password='root',
        host='localhost',
        port=3306,
        database='cs581_rideshare')    
    sql1 = "select pools_remaining from cs581_rideshare.user_preferences2 where tripId = %s" %trip_id
    cursor1 = cnn.cursor()
    cursor1.execute(sql1)
    result = cursor1.fetchall()
    cursor1.close()
    if result[0][0] > 0:
        res=result[0][0]
        res-=1
        sql = "update cs581_rideshare.user_preferences2 set pools_remaining=%s where tripId = %s "%(res,trip_id)
        cursor1 = cnn.cursor()
        cursor1.execute(sql)
        cnn.commit()
        cursor1.close()
        cnn.close()
        return True
    else:
        return False
    
def nextpool_list(global_nextpool):
    trip_list = []
    for abc in global_nextpool:
        trip = get_trip_details(abc)
        trip_list.append(trip)
    return trip_list

### Functions to find the potential ride shares from all the trips in the pool window
    

In [111]:

def get_graph(trips):
    graph_for_first_iteration = nx.Graph()
    distance_matrix = {}
    dm_individual = {}
    d2 = {}
    #print(distance_matrix)
    for i in range(len(trips)):
        for j in range(i, len(trips)):
            x = trips[i].id
            y = trips[j].id
            trip_distance_i = float(trips[i].gh_distance)
            trip_time_i = float(trips[i].gh_time)
            dm_individual[x] = trip_distance_i
            trip_distance_j = float(trips[j].gh_distance)
            trip_time_j = float(trips[j].gh_time)
            dm_individual[y] = trip_distance_j
            distance_between_destinations,time_between_destinations = GraphHopperDistance.find_distance(trips[i].dropoff_lat, trips[i].dropoff_long, trips[j].dropoff_lat, trips[j].dropoff_long)
            d2[y]=distance_between_destinations
            a, b = sorted([trip_distance_i, trip_distance_j])
            ab = distance_between_destinations
            distance_for_combined_trip = a + ab
            #CHECK FOR TIME DELAY CONSTRAINTS
            if(trip_distance_i <= trip_distance_j):
                shared_distance = trip_distance_i
                treshold = 1.2*trip_time_j
                time_delay = trip_time_i + time_between_destinations
            else:
                shared_distance = trip_distance_j
                treshold = 1.2*trip_time_i
                time_delay = trip_time_j + time_between_destinations
            #Calculate Sharing Distance Gain
            sharing_gain = abs(distance_for_combined_trip-(a+b))
            
            #Check if Socially Similar
            xyz = areSociallyCompatible(x,y,shared_distance)
            
            #Include 'and xyz' in If condition if you want to check for social score
            if(time_delay < treshold):
                graph_for_first_iteration.add_nodes_from([trips[i].id, trips[j].id])
                graph_for_first_iteration.add_edge(trips[i].id, trips[j].id, weight=sharing_gain, delay=time_delay)
        distance_matrix[x]=copy.deepcopy(d2)
    return (graph_for_first_iteration,distance_matrix,dm_individual)

#This method is to run the maximum matching on the constructed graph and to get the results
def get_merged_trips(graph12):
    list_of_merged_trips = {}
    matched_edges = nx.algorithms.matching.max_weight_matching(graph12)
    for i in matched_edges:
        trip1 = i
        trip2 = matched_edges[i]
        list_of_merged_trips[Mergedtrips(trip1, trip2, )] = {"sharing_gain": graph12[trip1][trip2]["weight"], "delay": graph12[trip1][trip2]["delay"]}
    return list(list_of_merged_trips)

#remove edges for the second round
def remove_sharedtrip_edge_from_graph(graph, list_of_merged_trips):
    graph_copy = copy.deepcopy(graph)
    for merged_trip in list_of_merged_trips:
        graph_copy.remove_edge(merged_trip.trip_list1[0], merged_trip.trip_list1[1])
    return graph_copy

#This is to get the Lone trips after the algorithm finishes running
def single_trip(list_of_merged_trips):
    single_trip_list = []
    tripdetails = get_data(start_date,start_time,end_time)
    for trip in tripdetails:
        merged = False
        for merged_trip in list_of_merged_trips:
            if(trip.id in merged_trip.trip_list1):
                merged = True
                break
        if (not(merged)):
            single_trip_list.append(trip.id)
    return single_trip_list

#This method is to check of a method is already merged with another node
def if_merged(list_of_merged_trips, trip):
    for merged_trip in list_of_merged_trips:
        if(trip in merged_trip.trip_list1):
            return len(merged_trip.trip_list1) < 3
    return True


#This method is find a trip from a list of merged trips
def better_merges(list_of_merged_trips, merged_trip):
    try:
        def matched_trip(list_of_merged_trips, trip):
            for merged_trip in list_of_merged_trips:
                if(trip in merged_trip.trip_list1):
                    if(len(merged_trip.trip_list1) <= 2):
                        return merged_trip
        
        trip1 = matched_trip(list_of_merged_trips, merged_trip.trip_list1[0])
        trip2 = matched_trip(list_of_merged_trips, merged_trip.trip_list1[1])

        if trip1 and trip2:
            trip1.add(trip2.trip_list1[0])
            trip1.add(trip2.trip_list1[1])
            list_of_merged_trips.pop(list_of_merged_trips.index(trip2))
            return merged_trip

        elif trip1:
            if if_merged(list_of_merged_trips, merged_trip.trip_list1[1]):
                trip1.add(merged_trip.trip_list1[1])
            return trip1

        elif trip2:
            if if_merged(list_of_merged_trips, merged_trip.trip_list1[0]):
                trip2.add(merged_trip.trip_list1[0])
            return trip2
    except Exception as e:
        raise e

## Function to call all the other functions and find the overall merged trips for the given Start and End time

In [112]:
import logging
import networkx as nx
import copy
import datetime



#This is the main method for executing the algorithm
def algorithm(global_nextpool,start_date,start_time,end_time):
    trip_subset = get_data(start_date,start_time,end_time)
    print("trip_subset len:",len(trip_subset))
    list1 = nextpool_list(global_nextpool)
    trip_subset = trip_subset + list1
    print("new trip_subset len:",len(trip_subset))
    tx = datetime.datetime.now()
    graph_for_first_iteration,dist_matrix,dm_individual = get_graph(trip_subset)
    ty = datetime.datetime.now()
    print(ty - tx)
    trips_merged_in_first_round = get_merged_trips(graph_for_first_iteration)
    graph_for_second_iteration = remove_sharedtrip_edge_from_graph(graph_for_first_iteration, trips_merged_in_first_round)

    merged_trips = get_merged_trips(graph_for_second_iteration)
    for merged_trip in merged_trips:
        better_merges(trips_merged_in_first_round, merged_trip)

    actual_cost = 0.0
    saved_cost = 0.0
    actual_dist = 0.0
    saved_dist = 0.0
    cost = 0.0
    share_cost = 0.0
    dist = 0.0
    tripDist = 0.0
    four_trips = 0
    three_trips = 0
    two_trips = 0
    pool_detail = {}
    next_pool = []
    single_trips = single_trip(trips_merged_in_first_round)
    print("Single trips:")
    for trip in single_trips:
        print(trip)
        ac = trip
        if (nextpoolFunc(ac)):
            next_pool.append(trip)
    if(len(trip_subset)<2 and len(trip_subset)>0):
        tripdet = get_trip_details(trip_subset[0].id)
        dist12 = dist12 + float(tripdet.gh_distance)
    for i in trips_merged_in_first_round:
        if len(i.trip_list1) == 2:
            two_trips += 1
        elif len(i.trip_list1) == 3:
            three_trips += 1
        elif len(i.trip_list1) == 4:
            four_trips += 1
        actual_dist, shared_dist = i.saved_distance(dist_matrix,dm_individual)
        actual_cost, shared_cost = i.saved_cost(dm_individual)
        cost = cost + actual_cost
        share_cost = share_cost + shared_cost
        dist = dist + actual_dist
        tripDist = tripDist + float(shared_dist)
    if(len(trip_subset)<2):
        print ("Actual distance: ",dist12)
    else:
        print ("Actual distance: ",dist)
    #print ("Actual distance: ",dist)
    print ("Combined distance:",tripDist)
    print ("Actual Cost:",cost)
    print ("Combined Cost:",share_cost)
    print("single trips:", len(single_trips))
    print ("four_trips: ",four_trips)
    print ("two_trips:",two_trips)
    print ("three_trips:",three_trips)
    pool_detail = {'actual_dist':dist,'combined_dist':tripDist,'actual_cost':cost,'combined_cost':share_cost,'single_trips':len(single_trips),'two_sharing':two_trips,
                  'three_sharing':three_trips,'four_sharing':four_trips}
    return pool_detail,next_pool






## Method that calls the algorithm function

In [113]:

#This is to execute the algorithm when this method is made to run
if __name__ == "__main__":
    tic = datetime.datetime.now()
    pool_details = {} 
    global_nextpool = []
    while end_time <= until_time:
        print("\npool : ",start_time, "-",end_time)
        nextpool1 = []
        pool_details[str(start_time)],nextpool1 = algorithm(global_nextpool,start_date,start_time,end_time)
        global_nextpool = nextpool1
        start_time = time_plus(start_time,windowsize)
        end_time = time_plus(end_time,windowsize)
    print(pool_details)
    toc = datetime.datetime.now()
    print (toc - tic)


pool :  08:10:00 - 08:11:00
trip_subset len: 20
new trip_subset len: 20
0:00:04.879190
Single trips:
493325
493326
493330
493334
Actual distance:  82.65
Combined distance: 34.220000000000006
Actual Cost: 82.65
Combined Cost: 54.228
single trips: 4
four_trips:  3
two_trips: 2
three_trips: 0
{'08:10:00': {'two_sharing': 2, 'combined_cost': 54.228, 'three_sharing': 0, 'actual_cost': 82.65, 'single_trips': 4, 'combined_dist': 34.220000000000006, 'four_sharing': 3, 'actual_dist': 82.65}}
0:00:05.282466


**Writing the output dictionary to a csv file** 

In [ ]:
bx = []
for key,values in pool_details.items():
    ax = []
    ax.append(key)
    for abc in list(values.values()):
        ax.append(abc)
    bx.append(ax)
print(bx)
import csv
with open('output.csv', 'w') as csv_file:
    csvwriter = csv.writer(csv_file, delimiter=',')
    for row in bx:
        csvwriter.writerow(row)